<a href="https://colab.research.google.com/github/JoeElgendy/Amit-ML-DeepLearning-assignments/blob/main/Dogs_vs_Cats_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 

In [7]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"youssefelgendy9","key":"ed2050b91f5978b707146dfbe196d470"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats

 99% 800M/812M [00:16<00:00, 152MB/s]
100% 812M/812M [00:16<00:00, 52.3MB/s]


In [8]:
os.listdir()

['.config', 'dogs-vs-cats.zip', 'sample_data']

In [9]:
import zipfile


In [10]:
zip_files = ['test1', 'train']
zip_files = ['test1', 'train']
for zip_file in zip_files:
    with zipfile.ZipFile("/content/dogs-vs-cats.zip".format(zip_file),"r") as z:
        z.extractall(".")
        print("{} unzipped".format(zip_file))

test1 unzipped
train unzipped


In [11]:
os.listdir()

['.config',
 'train.zip',
 'test1.zip',
 'dogs-vs-cats.zip',
 'sampleSubmission.csv',
 'sample_data']

In [12]:
with zipfile.ZipFile("train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("test1.zip","r") as z:
    z.extractall(".")

In [13]:
image_path='/content'


In [15]:
#stating the path of the folder in google colab
IMAGE_FOLDER_PATH="/content/train"
#extracting all the files in the directory
FILE_NAMES=os.listdir(IMAGE_FOLDER_PATH)
WIDTH=150
HEIGHT=150

#creating two list one for the target and one for the full path
targets=list()
full_paths=list()
# thank god the image has the target type in its name before the '.'
for file_name in FILE_NAMES:
    target=file_name.split(".")[0]
    # to add the image name to the path
    full_path=os.path.join(IMAGE_FOLDER_PATH, file_name)
    full_paths.append(full_path)
    targets.append(target)

dataset=pd.DataFrame()
dataset['image_path']=full_paths
dataset['target']=targets

In [16]:
dataset.head(10)

,image_path,target
0,/content/train/dog.10084.jpg,dog
1,/content/train/dog.10174.jpg,dog
2,/content/train/dog.10910.jpg,dog
3,/content/train/cat.7996.jpg,cat
4,/content/train/dog.5696.jpg,dog
5,/content/train/cat.5943.jpg,cat
6,/content/train/cat.8905.jpg,cat
7,/content/train/cat.5486.jpg,cat
8,/content/train/dog.7655.jpg,dog
9,/content/train/cat.11139.jpg,cat


In [ ]:
target_counts=dataset['target'].value_counts()
print('number of dogs in the data : {}'.format(target_counts['dog']))
print('number of cats in the data : {}'.format(target_counts['cat']))

number of dogs in the data : 12500
number of cats in the data : 12500


In [ ]:
import cv2

In [3]:
from keras import layers 
from keras import models
from keras import optimizers 
from keras.preprocessing.image import ImageDataGenerator
from keras import applications 

from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn import metrics 

In [2]:
x_train,x_val=train_test_split(dataset,random_state=40,shuffle=True,test_size=0.2)

NameError: ignored

In [ ]:
x_train.shape

(20000, 2)

In [ ]:
x_train['target'].value_counts()

cat    10065
dog     9935
Name: target, dtype: int64

In [ ]:
x_val['target'].value_counts()

dog    2565
cat    2435
Name: target, dtype: int64

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(WIDTH,HEIGHT,3)))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f2521667e10>>

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 146, 146, 32)      9248      
                                                                 
 batch_normalization (BatchN  (None, 146, 146, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 73, 73, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 73, 73, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        1

In [ ]:
model.compile(loss="binary_crossentropy", 
             optimizer='adam',
             metrics=["accuracy"])

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
train_datagenerator=train_datagen.flow_from_dataframe(dataframe=x_train,x_col='image_path',y_col='target',target_size=(WIDTH,HEIGHT),class_mode='binary',batch_size=150)

Found 20000 validated image filenames belonging to 2 classes.


In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_datagenerator=test_datagen.flow_from_dataframe(dataframe=x_val,x_col='image_path',y_col='target',target_size=(WIDTH,HEIGHT),class_mode='binary',batch_size=150)

Found 5000 validated image filenames belonging to 2 classes.


In [ ]:
modelhistory1=model.fit_generator(train_datagenerator,epochs=50,validation_data=test_datagenerator,validation_steps=x_val.shape[0]//150,steps_per_epoch=x_train.shape[0]//150, verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/50
133/133 [==============================] - 171s 1s/step - loss: 0.7686 - accuracy: 0.6100 - val_loss: 1.5513 - val_accuracy: 0.4873
Epoch 2/50
133/133 [==============================] - 155s 1s/step - loss: 0.5964 - accuracy: 0.6811 - val_loss: 0.9240 - val_accuracy: 0.5273
Epoch 3/50
133/133 [==============================] - 154s 1s/step - loss: 0.5374 - accuracy: 0.7292 - val_loss: 1.2753 - val_accuracy: 0.5154
Epoch 4/50
133/133 [==============================] - 154s 1s/step - loss: 0.4970 - accuracy: 0.7597 - val_loss: 0.8148 - val_accuracy: 0.6303
Epoch 5/50
133/133 [==============================] - 155s 1s/step - loss: 0.4348 - accuracy: 0.8019 - val_loss: 0.4230 - val_accuracy: 0.8091
Epoch 6/50
133/133 [==============================] - 155s 1s/step - loss: 0.3876 - accuracy: 0.8251 - val_loss: 0.4527 - val_accuracy: 0.7943
Epoch 7/50
133/133 [==============================] - 155s 1s/step - loss: 0.3425 - accuracy: 0.8469 - val_loss: 0.3766 - val_accuracy: 0.8212

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
model.save_weights('/content/gdrive')

In [ ]:
model.save('final_modellll_noo.h5')

# Transfer Learning 

In [ ]:
from tensorflow import keras 
from functools import partial 
import numpy as np 

In [ ]:
img_size=224 
input_size=(img_size,img_size,3)

In [ ]:
epochs=10 
batch_size=64

pre_trained_model=keras.applications.VGG19(input_shape=input_size,include_top=False,weights="imagenet")
for layer in pre_trained_model.layers[:15]:
  layer.trainable=False 
for layer in pre_trained_model.layers[15:]:
  layer.trainable=True 


In [ ]:
pre_trained_model.summary()

In [ ]:
output_layer=pre_trained_model.get_layer('block5_pool')
output=output_layer.output

In [ ]:
x=keras.layers.GlobalMaxPool2D()(output)
x=keras.layers.Dense(512,activation='relu')(x)
x=keras.layers.Dropout(0.5)(x)
x=keras.layers.Dense(1,activation='sigmoid')(x)
model=keras.models.Model(pre_trained_model.input,x)
model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.SGD(lr=1e-4,momentum=0.9),metrics=['accuracy'])
model.summary()

In [ ]:
history_pre_trained=pre_trained_model.fit_generator(train_datagenerator,epochs=epochs,validation_data=test_datagenerator,validation_steps=x_val.shape[0]//150,steps_per_epoch=x_train.shape[0]//150, verbose=1)

Transfer Learning using mobilenet